### Step 1: Data Exploration and Preprocessing

#### 1. Load the Dataset
First, we'll load the dataset from the Hugging Face repository using the `datasets` library.

In [ ]:
import os
huggingface_token = os.getenv("HF_TOKEN")

In [ ]:
!pip install datasets

from datasets import load_dataset

# Load the legal summarization dataset
dataset = load_dataset("lighteval/legal_summarization", "BillSum", split="train")

# Display dataset info
print(dataset.info)


#### 2. Data Exploration
Explore the dataset to understand its structure and contents.

In [ ]:
# Display first few examples
print(dataset[0])

# Display dataset size
print("Train dataset size:", len(dataset))


#### 3. Data Preprocessing
Preprocess the text data by tokenizing, cleaning, and encoding it for model input.

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# Define a function for preprocessing
def preprocess_data(example):
    # Tokenize and encode the text
    inputs = tokenizer(example['article'], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    # Tokenize and encode the summary
    targets = tokenizer(example['summary'], padding='max_length', truncation=True, max_length=128, return_tensors='pt')
    return {'input_ids': inputs['input_ids'].squeeze(), 'labels': targets['input_ids'].squeeze()}

# Convert the training dataset to a format suitable for preprocessing
train_dataset = Dataset.from_dict({'article': dataset['article'], 'summary': dataset['summary']})

# Apply preprocessing to the training dataset
train_dataset = train_dataset.map(preprocess_data)

# Display preprocessed data sample
print(train_dataset[0])


### Step 2: Model Selection and Training

In [ ]:
!pip install accelerate==0.21.0
!pip install transformers[torch] -U

##### T5-small model will be used to train the dataset.

In [ ]:
!pip install transformers[torch] -U
!pip install accelerate -U

from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load the tokenizer and model (using the t5-small variant)
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Load the dataset
dataset = load_dataset("lighteval/legal_summarization", "BillSum", split="train")

# Define a function for data preprocessing
def preprocess_data(example):
    inputs = tokenizer(example['article'], padding='max_length', truncation=True, max_length=128, return_tensors='pt')
    targets = tokenizer(example['summary'], padding='max_length', truncation=True, max_length=32, return_tensors='pt')
    return {'input_ids': inputs['input_ids'].squeeze(), 'labels': targets['input_ids'].squeeze()}

# Apply preprocessing to the dataset
train_dataset = dataset.map(preprocess_data)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./legal_summarization_output",
    per_device_train_batch_size=4,  # Adjust batch size
    gradient_accumulation_steps=64,  # Increase accumulation steps
    num_train_epochs=1,              # Limit number of epochs
    save_steps=500,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

from google.colab import files

# Compress the fine-tuned model directory into a zip file
!zip -r fine_tuned_model.zip ./fine_tuned_model

# Download the zip file
files.download("fine_tuned_model.zip")


#### Uploaded model and other necessary files to goole drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
from google.colab import files
Uploaded = files.upload()

In [ ]:
!cp -r /content/fine_tuned_model /content/drive/MyDrive/
!cp -r /content/legal_summarization_output /content/drive/MyDrive/
!cp /content/fine_tuned_model.zip /content/drive/MyDrive/

# Skip when running outside Colab

#### ROUGE score calculation

In [ ]:
!pip install rouge-score

from datasets import load_dataset, load_metric
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
import numpy as np

# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("./fine_tuned_model")
model = T5ForConditionalGeneration.from_pretrained("./fine_tuned_model")

# Initialize the summarization pipeline, explicitly setting it to use CPU
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=-1)

# Define the metric
rouge_metric = load_metric("rouge")

# Load the test dataset
test_dataset = load_dataset("lighteval/legal_summarization", "BillSum", split="test")

# Evaluate on a subset to keep the execution manageable
num_samples = 100  # Adjust this number based on your needs and capabilities
subset = test_dataset.select(range(num_samples))

generated_summaries = []
references = []

for sample in subset:
    # Generate summary
    input_text = sample["article"]
    summary = summarizer(input_text, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)[0]["summary_text"]
    generated_summaries.append(summary)
    references.append(sample["summary"])

# Compute and print ROUGE scores
results = rouge_metric.compute(predictions=generated_summaries, references=references, use_stemmer=True)
for key in results.keys():
    print(f"{key}: {np.mean(results[key]):.4f}")


#### dataeset distribution

In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Given summary
given_summary = "Public Safety and Protection Investment Act of 2003 - Amends the Internal Revenue Code to allow businesses to expense the costs of purchasing and installing qualifying security devices."

# Generated summary
generated_summary = "Public Safety and Protection Investment Act of 2003 amended by inserting after the item relating to section 179A the following new item: Sec.179B. a taxpayer may elect to treat the cost of any qualifying security device as an expense which is not chargeable to capital account."

# Tokenize summaries
nltk.download('punkt')
given_tokens = nltk.word_tokenize(given_summary)
generated_tokens = nltk.word_tokenize(generated_summary)

# Remove stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
given_filtered_tokens = [word for word in given_tokens if word.lower() not in stop_words]
generated_filtered_tokens = [word for word in generated_tokens if word.lower() not in stop_words]

# Convert tokens to strings
given_text = ' '.join(given_filtered_tokens)
generated_text = ' '.join(generated_filtered_tokens)

# Calculate cosine similarity
vectorizer = CountVectorizer().fit_transform([given_text, generated_text])
similarity = cosine_similarity(vectorizer[0], vectorizer[1])[0][0]

print("Cosine Similarity between the given and generated summaries:", similarity)


In [ ]:
import matplotlib.pyplot as plt

# Dataset information
train_examples = 18949
test_examples = 3269

# Plotting the dataset distribution
plt.figure(figsize=(8, 6))
plt.bar(['Train', 'Test'], [train_examples, test_examples], color=['blue', 'green'])
plt.xlabel('Dataset Split')
plt.ylabel('Number of Examples')
plt.title('Distribution of Examples in Train and Test Splits')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset

# Load the train and test datasets
train_dataset = load_dataset("lighteval/legal_summarization", "BillSum", split="train")
test_dataset = load_dataset("lighteval/legal_summarization", "BillSum", split="test")

# Get the number of examples in train and test splits
train_size = len(train_dataset)
test_size = len(test_dataset)

# Create a matrix with the sizes of train and test datasets
dataset_sizes = [[train_size, 0], [0, test_size]]

# Plot heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(dataset_sizes, annot=True, fmt='d', cmap='Blues', xticklabels=['Train', 'Test'], yticklabels=['Train', 'Test'])
plt.xlabel('Dataset Split')
plt.ylabel('Dataset Split')
plt.title('Dataset Distribution: Train vs Test')
plt.show()

#### length of the dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("lighteval/legal_summarization", "BillSum")

# Get the lengths of all articles
article_lengths = [len(article.split()) for article in dataset["train"]["article"]]

# Plot histogram
plt.figure(figsize=(10, 6))
plt.hist(article_lengths, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
plt.title('Distribution of Article Lengths')
plt.xlabel('Length of Articles')
plt.ylabel('Number of Articles')
plt.grid(True)
plt.show()


In [ ]:
import seaborn as sns

# Create distribution plots
plt.figure(figsize=(10, 5))
sns.histplot([len(article) for article in train_dataset["article"]], color="skyblue", label="Articles")
sns.histplot([len(summary) for summary in train_dataset["summary"]], color="green", label="Summaries")
plt.title("Length Distribution of Articles and Summaries")
plt.xlabel("Length")
plt.ylabel("Count")
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Define a function to compute novelty scores
def compute_novelty_scores(articles, summaries):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix_articles = tfidf_vectorizer.fit_transform(articles)
    tfidf_matrix_summaries = tfidf_vectorizer.transform(summaries)

    # Compute cosine similarity between article and summary TF-IDF representations
    cosine_similarities = cosine_similarity(tfidf_matrix_articles, tfidf_matrix_summaries)

    # Derive novelty scores from cosine similarities
    novelty_scores = 1 - np.max(cosine_similarities, axis=1)

    return novelty_scores

# Compute novelty scores for each article-summary pair
novelty_scores = compute_novelty_scores(train_dataset["article"], train_dataset["summary"])

# Plot histogram of novelty scores
plt.figure(figsize=(8, 6))
plt.hist(novelty_scores, bins=20, color='skyblue', edgecolor='black')
plt.title('Novelty Score Distribution')
plt.xlabel('Novelty Score')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()



In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load pre-trained model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Article to summarize
article = "SECTION 1. LIABILITY OF BUSINESS ENTITIES PROVIDING USE OF FACILITIES TO NONPROFIT ORGANIZATIONS. (a) Definitions.--In this section: (1) Business entity.--The term ``business entity'' means a firm, corporation, association, partnership, consortium, joint venture, or other form of enterprise. (2) Facility.--The term ``facility'' means any real property, including any building, improvement, or appurtenance. (3) Gross negligence.--The term ``gross negligence'' means voluntary and conscious conduct by a person with knowledge (at the time of the conduct) that the conduct is likely to be harmful to the health or well-being of another person. (4) Intentional misconduct.--The term ``intentional misconduct'' means conduct by a person with knowledge (at the time of the conduct) that the conduct is harmful to the health or well-being of another person. (5) Nonprofit organization.--The term ``nonprofit organization'' means-- (A) any organization described in section 501(c)(3) of the Internal Revenue Code of 1986 and exempt from tax under section 501(a) of such Code; or (B) any not-for-profit organization organized and conducted for public benefit and operated primarily for charitable, civic, educational, religious, welfare, or health purposes. (6) State.--The term ``State'' means each of the several States, the District of Columbia, the Commonwealth of Puerto Rico, the Virgin Islands, Guam, American Samoa, the Northern Mariana Islands, any other territory or possession of the United States, or any political subdivision of any such State, territory, or possession. (b) Limitation on Liability.-- (1) In general.--Subject to subsection (c), a business entity shall not be subject to civil liability relating to any injury or death occurring at a facility of the business entity in connection with a use of such facility by a nonprofit organization if-- (A) the use occurs outside of the scope of business of the business entity; (B) such injury or death occurs during a period that such facility is used by the nonprofit organization; and (C) the business entity authorized the use of such facility by the nonprofit organization. (2) Application.--This subsection shall apply-- (A) with respect to civil liability under Federal and State law; and (B) regardless of whether a nonprofit organization pays for the use of a facility. (c) Exception for Liability.--Subsection (b) shall not apply to an injury or death that results from an act or omission of a business entity that constitutes gross negligence or intentional misconduct, including any misconduct that-- (1) constitutes a crime of violence (as that term is defined in section 16 of title 18, United States Code) or act of international terrorism (as that term is defined in section 2331 of title 18) for which the defendant has been convicted in any court; (2) constitutes a hate crime (as that term is used in the Hate Crime Statistics Act (28 U.S.C. 534 note)); (3) involves a sexual offense, as defined by applicable State law, for which the defendant has been convicted in any court; or (4) involves misconduct for which the defendant has been found to have violated a Federal or State civil rights law. (d) Superseding Provision.-- (1) In general.--Subject to paragraph (2) and subsection (e), this Act preempts the laws of any State to the extent that such laws are inconsistent with this Act, except that this Act shall not preempt any State law that provides additional protection from liability for a business entity for an injury or death with respect to which conditions under subparagraphs (A) through (C) of subsection (b)(1) apply. (2) Limitation.--Nothing in this Act shall be construed to supersede any Federal or State health or safety law. (e) Election of State Regarding Nonapplicability.--This Act shall not apply to any civil action in a State court against a business entity in which all parties are citizens of the State if such State enacts a statute-- (1) citing the authority of this subsection; (2) declaring the election of such State that this Act shall not apply to such civil action in the State; and (3) containing no other provision."
# Tokenize and encode the article
inputs = tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)

# Generate summary
summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

# Decode and print the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Generated Summary:", summary)


In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
import random

# Example articles and summaries (replace with your actual data)
articles = ["SECTION 1. LIABILITY OF BUSINESS ENTITIES PROVIDING USE OF FACILITIES TO NONPROFIT ORGANIZATIONS. (a) Definitions.--In this section: (1) Business entity.--The term ``business entity'' means a firm, corporation, association, partnership, consortium, joint venture, or other form of enterprise. (2) Facility.--The term ``facility'' means any real property, including any building, improvement, or appurtenance. (3) Gross negligence.--The term ``gross negligence'' means voluntary and conscious conduct by a person with knowledge (at the time of the conduct) that the conduct is likely to be harmful to the health or well-being of another person. (4) Intentional misconduct.--The term ``intentional misconduct'' means conduct by a person with knowledge (at the time of the conduct) that the conduct is harmful to the health or well-being of another person. (5) Nonprofit organization.--The term ``nonprofit organization'' means-- (A) any organization described in section 501(c)(3) of the Internal Revenue Code of 1986 and exempt from tax under section 501(a) of such Code; or (B) any not-for-profit organization organized and conducted for public benefit and operated primarily for charitable, civic, educational, religious, welfare, or health purposes. (6) State.--The term ``State'' means each of the several States, the District of Columbia, the Commonwealth of Puerto Rico, the Virgin Islands, Guam, American Samoa, the Northern Mariana Islands, any other territory or possession of the United States, or any political subdivision of any such State, territory, or possession. (b) Limitation on Liability.-- (1) In general.--Subject to subsection (c), a business entity shall not be subject to civil liability relating to any injury or death occurring at a facility of the business entity in connection with a use of such facility by a nonprofit organization if-- (A) the use occurs outside of the scope of business of the business entity; (B) such injury or death occurs during a period that such facility is used by the nonprofit organization; and (C) the business entity authorized the use of such facility by the nonprofit organization. (2) Application.--This subsection shall apply-- (A) with respect to civil liability under Federal and State law; and (B) regardless of whether a nonprofit organization pays for the use of a facility. (c) Exception for Liability.--Subsection (b) shall not apply to an injury or death that results from an act or omission of a business entity that constitutes gross negligence or intentional misconduct, including any misconduct that-- (1) constitutes a crime of violence (as that term is defined in section 16 of title 18, United States Code) or act of international terrorism (as that term is defined in section 2331 of title 18) for which the defendant has been convicted in any court; (2) constitutes a hate crime (as that term is used in the Hate Crime Statistics Act (28 U.S.C. 534 note)); (3) involves a sexual offense, as defined by applicable State law, for which the defendant has been convicted in any court; or (4) involves misconduct for which the defendant has been found to have violated a Federal or State civil rights law. (d) Superseding Provision.-- (1) In general.--Subject to paragraph (2) and subsection (e), this Act preempts the laws of any State to the extent that such laws are inconsistent with this Act, except that this Act shall not preempt any State law that provides additional protection from liability for a business entity for an injury or death with respect to which conditions under subparagraphs (A) through (C) of subsection (b)(1) apply. (2) Limitation.--Nothing in this Act shall be construed to supersede any Federal or State health or safety law. (e) Election of State Regarding Nonapplicability.--This Act shall not apply to any civil action in a State court against a business entity in which all parties are citizens of the State if such State enacts a statute-- (1) citing the authority of this subsection; (2) declaring the election of such State that this Act shall not apply to such civil action in the State; and (3) containing no other provision."]
summaries = ["Shields a business entity from civil liability relating to any injury or death occurring at a facility of that entity in connection with a use of such facility by a nonprofit organization if: (1) the use occurs outside the scope of business of the business entity; (2) such injury or death occurs during a period that such facility is used by such organization; and (3) the business entity authorized the use of such facility by the organization. Makes this Act inapplicable to an injury or death that results from an act or omission of a business entity that constitutes gross negligence or intentional misconduct, including misconduct that: (1) constitutes a hate crime or a crime of violence or act of international terrorism for which the defendant has been convicted in any court; or (2) involves a sexual offense for which the defendant has been convicted in any court or misconduct for which the defendant has been found to have violated a Federal or State civil rights law. Preempts State laws to the extent that such laws are inconsistent with this Act, except State law that provides additional protection from liability. Specifies that this Act shall not be construed to supersede any Federal or State health or safety law. Makes this Act inapplicable to any civil action in a State court against a business entity in which all parties are citizens of the State if such State, citing this Act's authority and containing no other provision, enacts a statute declaring the State's election that this Act shall not apply to such action in the State.", "the term 'business entity'' means a firm, corporation, association, partnership, consortium, joint venture, or other form of enterprise. the term 'facility'' means any real property, including any building, improvement, or appurtenance. the term 'gross negligence'' means voluntary and conscious conduct by a person with knowledge that the conduct is harmful to the health or well-being of another person."]

# Tokenize articles and summaries
article_tokens = [word.lower() for article in articles for word in article.split()]
summary_tokens = [word.lower() for summary in summaries for word in summary.split()]

# Compute keyword frequencies
article_keyword_freq = Counter(article_tokens)
summary_keyword_freq = Counter(summary_tokens)

# Get random 3 keywords for x-axis
random_article_keywords = random.sample(list(article_keyword_freq.keys()), min(3, len(article_keyword_freq)))
random_summary_keywords = random.sample(list(summary_keyword_freq.keys()), min(3, len(summary_keyword_freq)))

# Plot bar charts
plt.figure(figsize=(10, 5))
plt.bar(article_keyword_freq.keys(), article_keyword_freq.values(), alpha=0.5, label='Articles')
plt.bar(summary_keyword_freq.keys(), summary_keyword_freq.values(), alpha=0.5, label='Summaries')
plt.title("Keyword Frequency in Articles and Summaries")
plt.xlabel("Keywords")
plt.ylabel("Frequency")
plt.xticks(random_article_keywords + random_summary_keywords)  # Set x-ticks to random keywords
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the ROUGE metrics
metrics = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']

# Extract ROUGE scores
rouge_scores = {metric: results[metric].mid.fmeasure for metric in metrics}

# Define colors for each metric
colors = {'rouge1': 'blue', 'rouge2': 'green', 'rougeL': 'orange', 'rougeLsum': 'red'}

# Plot line chart with bar chart
plt.figure(figsize=(10, 5))
for i, (metric, score) in enumerate(rouge_scores.items()):
    plt.plot(i, score, marker='o', color=colors[metric], label=metric.upper() + ' Score')
    plt.bar(i, score, color=colors[metric], alpha=0.5)
plt.xticks(np.arange(len(metrics)), [metric.upper() for metric in metrics])
plt.title("ROUGE Scores")
plt.xlabel("Metrics")
plt.ylabel("Scores")
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate random frequency-based data for demonstration
np.random.seed(0)
num_instances = 100  # Number of instances
rouge_scores = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-Lsum']
frequency_data = {metric: np.random.randint(0, 10, num_instances) for metric in rouge_scores}

# Plotting individual histograms for each ROUGE metric
plt.figure(figsize=(12, 8))

for i, metric in enumerate(rouge_scores, start=1):
    plt.subplot(2, 2, i)
    plt.hist(frequency_data[metric], bins=range(11), color='skyblue', edgecolor='black', alpha=0.7)
    plt.title(metric)
    plt.xlabel('Score')
    plt.ylabel('Frequency')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Given ROUGE scores for train and test splits
rouge_scores_train = {'ROUGE-1': 0.3949, 'ROUGE-2': 0.1994, 'ROUGE-L': 0.2895, 'ROUGE-Lsum': 0.2891}
rouge_scores_test = {'ROUGE-1': 0.3642, 'ROUGE-2': 0.1773, 'ROUGE-L': 0.2724, 'ROUGE-Lsum': 0.2715}

# Create a matrix with the ROUGE scores for train and test datasets
rouge_scores_matrix = np.array([[rouge_scores_train['ROUGE-1'], rouge_scores_test['ROUGE-1']],
                                 [rouge_scores_train['ROUGE-2'], rouge_scores_test['ROUGE-2']],
                                 [rouge_scores_train['ROUGE-L'], rouge_scores_test['ROUGE-L']],
                                 [rouge_scores_train['ROUGE-Lsum'], rouge_scores_test['ROUGE-Lsum']]])

# Plot heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(rouge_scores_matrix, annot=True, fmt='.4f', cmap='Blues', xticklabels=['Train', 'Test'], yticklabels=['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-Lsum'])
plt.xlabel('Dataset Split')
plt.ylabel('ROUGE Metric')
plt.title('ROUGE Score Distribution: Train vs Test')
plt.show()


In [ ]:
!pip install rouge-score
from rouge_score import rouge_scorer

# Test 1
reference_summaries_1 = [
    "New Millennium Classrooms Act - Amends the Internal Revenue Code to: (1) expand the tax deduction for computer donations by corporations to tax-exempt schools to include donations to public libraries; (2) increase from two to three years the age of computers that can be contributed for such deduction; and (3) allow a business tax credit of 30 percent of the value of computers donated to tax-exempt schools and public libraries. Increases the amount of such credit to 50 percent for contributions to schools or public libraries in empowerment zones, enterprise communities, and Indian reservations."
]

generated_summary_1 = "the Internal Revenue Code of 1986 relates to contributions of computer technology and equipment for elementary or secondary school purposes. it is amended by striking or'' at the end of subclause (I), by inserting or'' at the end of subclause (II), and by inserting after subclause (II) the following new subclause: (i) a public library (within the meaning of section 213(2)A)"

# Test 2
reference_summaries_2 = [
    "Public Safety and Protection Investment Act of 2003 - Amends the Internal Revenue Code to allow businesses to expense the costs of purchasing and installing qualifying security devices."
]

generated_summary_2 = "Public Safety and Protection Investment Act of 2003 amended by inserting after the item relating to section 179A the following new item: Sec. 179B. a taxpayer may elect to treat the cost of any qualifying security device as an expense which is not chargeable to capital account."

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)

# Calculate ROUGE scores
scores_1 = scorer.score(reference_summaries_1[0], generated_summary_1)
scores_2 = scorer.score(reference_summaries_2[0], generated_summary_2)

# Print scores
print("Test 1 ROUGE Scores:", scores_1)
print("Test 2 ROUGE Scores:", scores_2)


### Step 4 : Model deployment

In [ ]:
!pip install streamlit -q

In [ ]:
# run this command which is indead the " " as it shows ip "!wget -q -O - ipv4.icanhazip.com"

In [ ]:
!pip install streamlit
!pip install PyMuPDF
!pip install transformers

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

In [ ]:
from google.colab import drive
drive.mount('/content/drive')